In [ ]:
### WIP



In [ ]:
# https://github.com/SeldonIO/alibi-detect#drift-detection

In [ ]:
!pip install alibi-detect
!pip install tensorflow

In [231]:
import pandas as pd
import numpy as np

In [232]:
data = pd.read_csv('../../data.csv')

# cd = MMDDrift(x_ref, backend='tensorflow', p_val=.05)
# preds = cd.predict(x)

In [233]:
data.head(5)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,148,Male,0,No,No,1,Yes,No,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,45.65,45.65,Yes
1,463,Male,0,Yes,Yes,4,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,Month-to-month,No,Electronic check,101.15,385.90,Yes
2,471,Female,1,No,No,17,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,20.65,330.60,No
3,496,Male,0,No,No,22,No,No phone service,DSL,No,...,Yes,No,No,Yes,One year,Yes,Bank transfer (automatic),43.75,903.60,Yes
4,833,Female,0,Yes,Yes,70,Yes,No,DSL,Yes,...,Yes,Yes,No,Yes,One year,No,Credit card (automatic),74.10,5222.30,No


In [378]:
# do all data massaging
# Convert binary variable into numeric so plotting is easier. We need to later take mean
data['Churn'] = data['Churn'].map({'Yes': 1, 'No': 0})

data.replace(" ", np.nan, inplace=True)

data['TotalCharges'] = pd.to_numeric(data['TotalCharges'])

mean = data['TotalCharges'].mean()
data.fillna(mean, inplace=True)

import category_encoders as ce
import joblib

names = ['gender', 'Partner', 'Dependents', 'PhoneService', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling']
# for column in names:
#     labelencoder(column)
data_enc = data
data_enc = data_enc.drop(['Churn', 'customerID'], axis=1)
enc = ce.ordinal.OrdinalEncoder(cols=names)
enc.fit(data_enc)
labelled_set = enc.transform(data_enc)

names = ['MultipleLines', 'InternetService', 'Contract', 'PaymentMethod', 'OnlineSecurity', 'OnlineBackup',
         'DeviceProtection', 'TechSupport']

ohe = ce.OneHotEncoder(cols=names)
data_ohe = data
data_ohe = data_ohe.drop(['Churn', 'customerID'], axis=1)
ohe.fit(data_ohe)
final_set = ohe.transform(labelled_set)

final_set.head(5)

/Users/faisalmasood/.pyenv/versions/3.7.5/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines_1,MultipleLines_2,MultipleLines_3,InternetService_1,...,Contract_1,Contract_2,Contract_3,PaperlessBilling,PaymentMethod_1,PaymentMethod_2,PaymentMethod_3,PaymentMethod_4,MonthlyCharges,TotalCharges
0,1,0,1,1,1,1,1,0,0,1,...,1,0,0,1,1,0,0,0,45.65,45.65
1,1,0,2,2,4,1,0,1,0,0,...,1,0,0,2,1,0,0,0,101.15,385.90
2,2,1,1,1,17,1,1,0,0,0,...,0,1,0,2,0,1,0,0,20.65,330.60
3,1,0,1,1,22,2,0,0,1,1,...,0,1,0,1,0,0,1,0,43.75,903.60
4,2,0,2,2,70,1,1,0,0,1,...,0,1,0,2,0,0,0,1,74.10,5222.30


In [235]:
from sklearn.model_selection import train_test_split
labels = data['Churn']
X_train, X_test, y_train, y_test = train_test_split(final_set, labels, test_size=0.2)
print ('Training Data Shape',X_train.shape, y_train.shape)
print ('Testing Data Shape',X_test.shape, y_test.shape)

Y = data['Churn']
X = final_set


Training Data Shape (5634, 36) (5634,)
Testing Data Shape (1409, 36) (1409,)


In [236]:
print(X_train.shape[1])


print(X_train.shape)

36
(5634, 36)


In [253]:
from alibi_detect.od import IForest
od = IForest(threshold=0.05, n_estimators=500)

od.fit(X_train.values)

In [255]:
# od_preds = od.predict(X_test, return_instance_score=True)
# print(od_preds)
#
#col #34 is the monthly bill amount
input_with_insane_monthly_charges = X_test.iloc[0]
# input_with_insane_monthly_charges.values[34] = 22975.0
# input_with_insane_monthly_charges.values[35] = 25000.0
print(input_with_insane_monthly_charges.values.reshape(1,36))

od_preds = od.predict(input_with_insane_monthly_charges.values.reshape(1,36), return_instance_score=True)
print(od_preds)

[[2.0000e+00 0.0000e+00 1.0000e+00 1.0000e+00 1.9000e+01 1.0000e+00
  0.0000e+00 1.0000e+00 0.0000e+00 1.0000e+00 0.0000e+00 0.0000e+00
  1.0000e+00 0.0000e+00 0.0000e+00 1.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 1.0000e+00 0.0000e+00 1.0000e+00 0.0000e+00 0.0000e+00
  1.0000e+00 1.0000e+00 1.0000e+00 0.0000e+00 0.0000e+00 1.0000e+00
  0.0000e+00 1.0000e+00 0.0000e+00 0.0000e+00 2.2975e+04 2.5000e+04]]
{'data': {'instance_score': array([0.05714478]), 'feature_score': None, 'is_outlier': array([1])}, 'meta': {'name': 'IForest', 'detector_type': 'offline', 'data_type': 'tabular'}}


In [363]:
# from tensorflow.keras.models import Sequential
# from alibi_detect.models.losses import elbo

from tensorflow.keras.layers import Dense, InputLayer, Input
import tensorflow as tf


n_features = X_train.shape[1]
latent_dim=7

encoder_net = tf.keras.Sequential(
    [
        InputLayer(input_shape=(n_features, ), name="encode1"),
        Dense(27, activation=tf.nn.relu,  name="encode2"),
        Dense(18, activation=tf.nn.relu,  name="encode3"),
        Dense(latent_dim, activation=tf.nn.relu, name="encode4")
    ])


decoder_net = tf.keras.Sequential(
    [
        InputLayer(input_shape=(latent_dim,), name="decode1"),
        Dense(18, activation=tf.nn.relu, name="decode2"),
        Dense(27, activation=tf.nn.relu, name="decode3"),
        Dense(n_features, activation=None, name="decode4")
    ])

In [374]:
from alibi_detect.od import OutlierVAE, OutlierAE

outlier_predictor = OutlierAE(
    threshold=10,
    encoder_net=encoder_net,
    decoder_net=decoder_net,
    data_type="tabular"
)

## Todo checl with VAE too
# outlier_predictor = OutlierVAE(
#     threshold=0.05,
#     score_type="mse",
#     encoder_net=encoder_net,
#     decoder_net=decoder_net,
#     latent_dim=latent_dim,
#     samples=20
# )

In [382]:
outlier_predictor.fit(X_train,  epochs=20, batch_size=24, verbose=True)

235/235 [=] - 2s 9ms/step - loss_ma: 0.5611

In [380]:
input_with_insane_monthly_charges = X_test.iloc[0]
input_with_insane_monthly_charges.values[34] = 22975.0
input_with_insane_monthly_charges.values[35] = 125000.0


preds = outlier_predictor.predict(np.reshape(input_with_insane_monthly_charges.values, (-1, 36)), outlier_type='instance',
                                   return_feature_score=True,  return_instance_score=True)
print(preds)


{'data': {'instance_score': array([281861.37473679]), 'feature_score': array([[6.39333262e+04, 1.57388793e+03, 3.22518133e+03, 1.39801822e+05,
        3.59810327e+06, 9.57559422e+03, 2.39024323e+03, 1.14104894e+05,
        4.40632282e+04, 4.60202081e+03, 6.13876461e+04, 2.02735996e+04,
        2.22199655e+04, 4.02135261e+05, 5.01090233e+04, 7.47303533e+04,
        1.65809516e+04, 6.28629522e+04, 2.82284606e+05, 4.21370873e+05,
        1.46301884e+05, 1.11369080e+05, 6.50528421e+03, 1.16749138e+05,
        1.01635770e+03, 1.45100453e+04, 2.71860404e+05, 2.59641882e+03,
        2.79192657e+04, 3.15601601e+04, 1.84062946e+04, 3.01033124e+05,
        1.71473154e+03, 9.51243918e+04, 3.58745796e+06, 1.75562500e+04]]), 'is_outlier': array([1])}, 'meta': {'name': 'OutlierAE', 'detector_type': 'offline', 'data_type': 'tabular'}}
